## Import Packages

In [57]:
import numpy as np
from PIL import Image
from os.path import getsize

## (a) implement the simplified DCT compression process



### Functions

In [2]:
# Define quantization tables 
Q = np.array([[8, 6, 6, 7, 6, 5, 8, 7], 
              [7, 7, 9, 9, 8, 10, 12, 20], 
        [13, 12, 11, 11, 12, 25, 18, 19], 
        [15, 20, 29, 26, 31, 30, 29, 26],
        [28, 28, 32, 36, 46, 39, 32, 34], 
        [44, 35, 28, 28, 40, 55, 41, 44], 
        [48, 49, 52, 52, 52, 31, 39, 57], 
        [61, 56, 50, 60, 46, 51, 52, 50]])

In [3]:
def dct2(block):
    """Compute 2D DCT of 8x8 block"""
    result = np.zeros((8, 8))
    for u in range(8):
        for v in range(8):
            # Compute C(u) and C(v) coefficients
            cu = 1 / np.sqrt(2) if u == 0 else 1
            cv = 1 / np.sqrt(2) if v == 0 else 1
            
            s = 0
            for x in range(8):
                for y in range(8):
                    s += block[x, y] * np.cos((2*x + 1)*u*np.pi / 16) * np.cos((2*y + 1)*v*np.pi / 16)
            
            result[u, v] = (1/4) * cu * cv * s
    return result

def uniformQ(rq, gq, bq, m):
    r_delta = (np.max(rq) - np.min(rq)) / (2**m - 1)  
    r_coef = np.round(rq / r_delta)
    g_delta = (np.max(gq) - np.min(gq)) / (2**m - 1)
    g_coef = np.round(gq / g_delta)
    b_delta = (np.max(bq) - np.min(bq)) / (2**m - 1)
    b_coef = np.round(bq / b_delta)

    compressed_blocks = r_coef, g_coef, b_coef
    delta = [r_delta, g_delta, b_delta]

    return compressed_blocks, delta

def dct_compress(image, n, m):
    size = image.shape[0]
    rq, gq, bq = np.zeros((size, size)), np.zeros((size, size)), np.zeros((size, size))
    numBlocks = int(image.shape[0]/8)

    for i in range(numBlocks):
      for j in range(numBlocks):
        rDct, gDct, bDct = np.zeros((8, 8)), np.zeros((8, 8)), np.zeros((8, 8)) 

        rBlock = image[i*8:i*8+8, j*8:j*8+8, 0] 
        rDct = dct2(rBlock)
        rq[i*8:i*8+n, j*8:j*8+n] = rDct[:n, :n]
        rq[i*8:i*8+8, j*8:j*8+8] = np.round(rq[i*8:i*8+8, j*8:j*8+8] / Q)
    
        gBlock = image[i*8:i*8+8, j*8:j*8+8, 1] 
        gDct = dct2(gBlock)
        gq[i*8:i*8+n, j*8:j*8+n] = gDct[:n, :n]
        gq[i*8:i*8+8, j*8:j*8+8] = np.round(gq[i*8:i*8+8, j*8:j*8+8] / Q)

        bBlock = image[i*8:i*8+8, j*8:j*8+8, 2] 
        bDct= dct2(bBlock)
        bq[i*8:i*8+n, j*8:j*8+n] = bDct[:n, :n]
        bq[i*8:i*8+8, j*8:j*8+8] = np.round(bq[i*8:i*8+8, j*8:j*8+8] / Q)

    # Use uniform Quantization
    compressed_blocks, delta = uniformQ(rq, gq, bq, m)

    return compressed_blocks, delta

In [4]:
def idct2(coefficients):
    """Compute 2D IDCT of 8x8 block"""
    result = np.zeros((8, 8))
    for x in range(8):
        for y in range(8):
            # Compute sum over frequency domain
            s = 0
            for u in range(8):
                for v in range(8):
                    # Compute C(u) and C(v) coefficients
                    cu = 1 / np.sqrt(2) if u == 0 else 1
                    cv = 1 / np.sqrt(2) if v == 0 else 1
                    # Compute final pixel value
                    s += cu * cv * coefficients[u, v] * np.cos((2*x + 1)*u*np.pi / 16) * np.cos((2*y + 1)*v*np.pi / 16)
            # Add 128 to restore original pixel value
            result[x, y] = (1/4) * s

    return result

def unquantization(compressed_blocks, delta):
    r_coef, g_coef, b_coef = compressed_blocks[0], compressed_blocks[1], compressed_blocks[2]
    rq = r_coef * delta[0]
    gq = g_coef * delta[1]
    bq = b_coef * delta[2]

    return rq, gq, bq

def dct_decompress(compressed_blocks, delta, n):
    rq, gq, bq = unquantization(compressed_blocks, delta)
    size = rq.shape[0]
    image = np.zeros((size, size, 3))
    rBlock, gBlock, bBlock = np.zeros((size, size)), np.zeros((size, size)), np.zeros((size, size))
    numBlocks = int(image.shape[0]/8)
    
    for i in range(numBlocks):
        for j in range(numBlocks):
            rDct = np.zeros((8, 8))
            gDct = np.zeros((8, 8))
            bDct = np.zeros((8, 8))

            rTmp = rq[i*8:i*8+8, j*8:j*8+8] * Q
            rDct[:n, :n] = rTmp[:n, :n]
            rBlock = idct2(rDct) 
        
            gTmp = gq[i*8:i*8+8, j*8:j*8+8] * Q
            gDct[:n, :n] = gTmp[:n, :n]
            gBlock = idct2(gDct)

            bTmp = bq[i*8:i*8+8, j*8:j*8+8] * Q
            bDct[:n, :n] = bTmp[:n, :n]
            bBlock = idct2(bDct) 
        
            image[i*8:i*8+8, j*8:j*8+8, 0] = rBlock
            image[i*8:i*8+8, j*8:j*8+8, 1] = gBlock
            image[i*8:i*8+8, j*8:j*8+8, 2] = bBlock

    return image


In [5]:
def PSNR(old, new):
    img = np.array(Image.open(old))
    new_img = np.array(Image.open(new))
    mse = np.mean((img.astype(np.float64) / 255 - new_img.astype(np.float64) / 255) ** 2)
    return 10 * np.log10(1. / mse)


### Tasks

In [26]:
input_path = 'Barbara.jpg'
output_path = ['output/bar_n2m4_a.jpg', 'output/bar_n2m8_a.jpg', 
               'output/bar_n4m4_a.jpg', 'output/bar_n4m8_a.jpg']
parameters = [[2, 4], [2, 8], [4, 4], [4, 8]]

for i in range(4):
    img = Image.open(input_path)
    img_arr = np.array(img)

    n, m = parameters[i]
    compressed_blocks, delta = dct_compress(img_arr, n, m)
    new_img_arr = dct_decompress(compressed_blocks, delta, n)
    new_img_arr = np.clip(new_img_arr, 0, 255).astype(np.uint8)
    new_img = Image.fromarray(new_img_arr)

    # Save the output image
    new_img.save(output_path[i])
    print(output_path[i])

print(input_path, ' task finished.')
        

output/bar_n2m4_a.jpg
output/bar_n2m8_a.jpg
output/bar_n4m4_a.jpg
output/bar_n4m8_a.jpg
Barbara.jpg  task finished.


In [5]:
input_path = 'Barbara.jpg'
output_path = ['output/bar_n2m4_a.jpg', 'output/bar_n2m8_a.jpg', 
               'output/bar_n4m4_a.jpg', 'output/bar_n4m8_a.jpg']
parameters = [[2, 4], [2, 8], [4, 4], [4, 8]]

for i in range(4):
    n, m = parameters[i]
    ratio = getsize(input_path) / getsize(output_path[i])
    before_bits = 512 * 512 * (8 * 3)
    after_bits = (512/8 * 512/8 * (n**2) * m * 3)
    print('compression ratio (n', n, 'm', m, ') =', before_bits/after_bits)
    print('PSNR = ', PSNR(input_path, output_path[i]))

compression ratio (n 2 m 4 ) = 32.0
PSNR =  24.375826022329154
compression ratio (n 2 m 8 ) = 16.0
PSNR =  25.558081884303046
compression ratio (n 4 m 4 ) = 8.0
PSNR =  25.11202255586902
compression ratio (n 4 m 8 ) = 4.0
PSNR =  28.85368062028883


In [28]:
input_path = 'cat.jpg'
output_path = ['output/cat_n2m4_a.jpg', 'output/cat_n2m8_a.jpg', 
               'output/cat_n4m4_a.jpg', 'output/cat_n4m8_a.jpg']
parameters = [[2, 4], [2, 8], [4, 4], [4, 8]]

for i in range(4):
    img = Image.open(input_path)
    img_arr = np.array(img)

    n, m = parameters[i]
    compressed_blocks, delta = dct_compress(img_arr, n, m)
    new_img_arr = dct_decompress(compressed_blocks, delta, n)
    new_img_arr = np.clip(new_img_arr, 0, 255).astype(np.uint8)
    new_img = Image.fromarray(new_img_arr)

    # Save the output image
    new_img.save(output_path[i])
    print(output_path[i])

print(input_path, ' task finished.')     

output/cat_n2m4_a.jpg
output/cat_n2m8_a.jpg
output/cat_n4m4_a.jpg
output/cat_n4m8_a.jpg
cat.jpg  task finished.


In [6]:
# compute compression raios and the PSNR
input_path = 'cat.jpg'
output_path = ['output/cat_n2m4_a.jpg', 'output/cat_n2m8_a.jpg', 
               'output/cat_n4m4_a.jpg', 'output/cat_n4m8_a.jpg']
parameters = [[2, 4], [2, 8], [4, 4], [4, 8]]

for i in range(4):
    n, m = parameters[i]
    ratio = getsize(input_path) / getsize(output_path[i])
    before_bits = 512 * 512 * (8 * 3)
    after_bits = (512/8 * 512/8 * (n**2) * m * 3)
    print('compression ratio (n', n, 'm', m, ') =', before_bits/after_bits)
    print('PSNR = ', PSNR(input_path, output_path[i]))

compression ratio (n 2 m 4 ) = 32.0
PSNR =  20.39847233872127
compression ratio (n 2 m 8 ) = 16.0
PSNR =  20.974389301040798
compression ratio (n 4 m 4 ) = 8.0
PSNR =  22.979456442361148
compression ratio (n 4 m 8 ) = 4.0
PSNR =  27.192311186618983


## (b) use the same process in (a) with image transformed to YCbCr color space with 4:2:0 chrominance subsampling

### Functions

In [63]:
rgb2ycbcr = np.array([[0.257, 0.504, 0.098], 
                      [-0.148, -0.291, 0.439], 
                      [0.439, -0.368, -0.071]])

def subsampling(ycbcr_arr):
    size = int(ycbcr_arr.shape[0]/2)
    sampled_cb = np.zeros((size, size))
    sampled_cb = ycbcr_arr[::2, ::2, 1]

    sampled_cr = np.zeros((size, size))
    sampled_cr = ycbcr_arr[::2, ::2, 2]

    return ycbcr_arr[:, :, 0], sampled_cb, sampled_cr

In [62]:
# Define quantization tables 
QL = np.array([[16, 11, 10, 16, 24, 40, 51, 61], 
                [12, 12, 14, 19, 26, 58, 60, 55], 
                [14, 13, 16, 24, 40, 57, 69, 56], 
                [14, 17, 22, 29, 51, 87, 80, 62],
                [18, 22, 37, 56, 68, 109, 103, 77], 
                [24, 36, 55, 64, 81, 104, 113, 92], 
                [49, 64, 78, 87, 103, 121, 120, 101], 
                [72, 92, 95, 98, 112, 100, 103, 99]])
QC = np.array([[17, 18, 24, 47, 99, 99, 99, 99], 
               [18, 21, 26, 66, 99, 99, 99, 99], 
               [24, 26, 56, 99, 99, 99, 99, 99], 
               [47, 66, 99, 99, 99, 99, 99, 99], 
               [99, 99, 99, 99, 99, 99, 99, 99],
               [99, 99, 99, 99, 99, 99, 99, 99],
               [99, 99, 99, 99, 99, 99, 99, 99],
               [99, 99, 99, 99, 99, 99, 99, 99]])

In [61]:
def uniformQ_sub(rq, gq, bq, m):
    r_delta = (np.max(rq) - np.min(rq)) / (2**m - 1)  
    r_coef = np.round(rq / r_delta)
    g_delta = (np.max(gq) - np.min(gq)) / (2**m - 1)
    g_coef = np.round(gq / g_delta)
    b_delta = (np.max(bq) - np.min(bq)) / (2**m - 1)
    b_coef = np.round(bq / b_delta)

    coef = [r_coef, g_coef, b_coef]
    delta = [r_delta, g_delta, b_delta]
    return coef, delta

In [55]:
def dct_compress_sub(Y, Cb, Cr, n, m):
    yq = np.zeros((Y.shape[0], Y.shape[1]))
    cbq = np.zeros((Cb.shape[0], Cb.shape[1]))
    crq = np.zeros((Cr.shape[0], Cr.shape[1]))

    numBlocks = int(Y.shape[0]/8)
    for i in range(numBlocks):
      for j in range(numBlocks):
        yDct = np.zeros((8, 8))

        yBlock = Y[i*8:i*8+8, j*8:j*8+8]
        yDct = dct2(yBlock)
        yq[i*8:i*8+n, j*8:j*8+n] = yDct[:n, :n]
        yq[i*8:i*8+8, j*8:j*8+8] = np.round(yq[i*8:i*8+8, j*8:j*8+8] / QL)


    numBlocks = int(Cb.shape[0]/8)
    for i in range(numBlocks):
      for j in range(numBlocks):
        cbDct, crDct = np.zeros((8, 8)), np.zeros((8, 8))
    
        cbBlock = Cb[i*8:i*8+8, j*8:j*8+8] 
        cbDct = dct2(cbBlock)
        cbq[i*8:i*8+n, j*8:j*8+n] = cbDct[:n, :n]
        cbq[i*8:i*8+8, j*8:j*8+8] = np.round(cbq[i*8:i*8+8, j*8:j*8+8] / QC)

        crBlock = Cr[i*8:i*8+8, j*8:j*8+8] 
        crDct= dct2(crBlock)
        crq[i*8:i*8+n, j*8:j*8+n] = crDct[:n, :n]
        crq[i*8:i*8+8, j*8:j*8+8] = np.round(crq[i*8:i*8+8, j*8:j*8+8] / QC)

    # Use uniform Quantization
    coef, delta = uniformQ_sub(yq, crq, cbq, m)
    #coef = [yq, cbq, crq]
    
    return coef, delta

In [60]:
def unquantization_sub(coef, delta):
    yq = coef[0] * delta[0]
    cbq = coef[1] * delta[1]
    crq = coef[2] * delta[2]

    return yq, cbq, crq

def rescale(sub):
    size = sub.shape[0]
    all = np.zeros((size*2, size*2))
    for i in range(size):
        for j in range(size):
            all[i*2:i*2+2, j*2:j*2+2] = sub[i, j]
    return all

In [56]:
def dct_decompress_sub(coef, delta, n):
    yq, cbq, crq = unquantization_sub(coef, delta)
    #yq, cbq, crq = coef[0], coef[1], coef[2]
    size = yq.shape[0]
    image = np.zeros((size, size, 3))

    numBlocks = int(image.shape[0]/8)
    for i in range(numBlocks):
        for j in range(numBlocks):
            yDct, yBlock = np.zeros((8, 8)), np.zeros((8, 8))

            yTmp = yq[i*8:i*8+8, j*8:j*8+8] * QL
            yDct[:n, :n] = yTmp[:n, :n]
            yBlock = idct2(yDct) 
        
            image[i*8:i*8+8, j*8:j*8+8, 0] = yBlock

    numBlocks = int(cbq.shape[0]/8)

    for i in range(numBlocks):
        for j in range(numBlocks):
            cbDct = np.zeros((8, 8))
            crDct = np.zeros((8, 8))
        
            cbTmp = cbq[i*8:i*8+8, j*8:j*8+8] * QC
            cbDct[:n, :n] = cbTmp[:n, :n]
            cbDct = idct2(cbDct)
            image[i*16:i*16+16, j*16:j*16+16, 1] = rescale(cbDct)

            crTmp = crq[i*8:i*8+8, j*8:j*8+8] * QC
            crDct[:n, :n] = crTmp[:n, :n]
            crDct = idct2(crDct) 
            image[i*16:i*16+16, j*16:j*16+16, 2] = rescale(crDct)
    
    return image

### Tasks

In [65]:
input_path = 'Barbara.jpg'
output_path = ['output/bar_n2m4_b.jpg', 'output/bar_n2m8_b.jpg', 
               'output/bar_n4m4_b.jpg', 'output/bar_n4m8_b.jpg']
parameters = [[2, 4], [2, 8], [4, 4], [4, 8]]

for i in range(1):
    img = Image.open(input_path)
    img_arr = np.array(img)
    originalShape = img_arr.shape
    ycbcr_arr = np.dot(img_arr.reshape(-1, 3), rgb2ycbcr.transpose()).reshape(originalShape)
    ycbcr_arr[:, :, 0] -= 16
    ycbcr_arr[:, :, 1] -= 128
    ycbcr_arr[:, :, 2] -= 128
    
    sampleY, sampleCb, sampleCr = subsampling(ycbcr_arr)
    
    n, m = parameters[i]
    coef, delta = dct_compress_sub(sampleY, sampleCb, sampleCr, n, m)
    print('Y:', np.unique(coef[0]))
    print('Cb:', np.unique(coef[1]))
    print('Cr:', np.unique(coef[2]))
    new_img_arr = dct_decompress_sub(coef, delta, n)

    # YCbCr to RGB

    new_img_arr[:, :, 0] += 16
    new_img_arr[:, :, 1] += 128
    new_img_arr[:, :, 2] += 128
   
    rgb_arr = np.dot(new_img_arr.reshape(-1,3), np.linalg.inv(rgb2ycbcr).transpose()).reshape(originalShape)
    rgb_arr = np.clip(rgb_arr, 0, 255).astype(np.uint8)
    new_img = Image.fromarray(rgb_arr)

    # Save the output image
    new_img.save(output_path[i])
    print(output_path[i])

print(input_path, ' task finished.')    

KeyboardInterrupt: 

In [64]:
# compute compression raios and the PSNR
input_path = 'Barbara.jpg'
output_path = ['output/bar_n2m4_b.jpg', 'output/bar_n2m8_b.jpg', 
               'output/bar_n4m4_b.jpg', 'output/bar_n4m8_b.jpg']
parameters = [[2, 4], [2, 8], [4, 4], [4, 8]]

for i in range(1):
    n, m = parameters[i]
    ratio = getsize(input_path) / getsize(output_path[i])
    before_bits = 512 * 512 * (8 * 3)
    after_bits = (512/8 * 512/8 * (n**2) * m) + (256/8 * 256/8 * (n**2) * m * 2)
    print('compression ratio (n', n, 'm', m, ') =', before_bits/after_bits)
    print('PSNR = ', PSNR(input_path, output_path[i]))

compression ratio (n 2 m 4 ) = 64.0
PSNR =  14.386675527287071


In [28]:
input_path = 'cat.jpg'
output_path = ['output/cat_n2m4_b.jpg', 'output/cat_n2m8_b.jpg', 
               'output/cat_n4m4_b.jpg', 'output/cat_n4m8_b.jpg']
parameters = [[2, 4], [2, 8], [4, 4], [4, 8]]

for i in range(4):
    img = Image.open(input_path)
    img_arr = np.array(img)
    originalShape = img_arr.shape
    ycbcr_arr = np.dot(img_arr.reshape(-1, 3), rgb2ycbcr.transpose()).reshape(originalShape)
    ycbcr_arr[:, :, 0] -= 16
    ycbcr_arr[:, :, 1] -= 128
    ycbcr_arr[:, :, 2] -= 128
    
    subsample_arr = subsampling(ycbcr_arr)
    
    n, m = parameters[i]
    compressed_blocks, delta = dct_compress_sub(subsample_arr, n, m)
    new_img_arr = dct_decompress_sub(compressed_blocks, delta, n)

    # YCbCr to RGB
    new_img_arr[:, :, 0] += 16
    new_img_arr[:, :, 1] += 128
    new_img_arr[:, :, 2] += 128
    rgb_arr = np.dot(new_img_arr.reshape(-1,3), np.linalg.inv(rgb2ycbcr).transpose()).reshape(originalShape)
    rgb_arr = np.clip(rgb_arr, 0, 255).astype(np.uint8)
    new_img = Image.fromarray(rgb_arr)

    # Save the output image
    new_img.save(output_path[i])
    print(output_path[i])

print(input_path, ' task finished.')    

output/cat_n2m4_b.jpg
output/cat_n2m8_b.jpg
output/cat_n4m4_b.jpg
output/cat_n4m8_b.jpg
cat.jpg  task finished.


In [29]:
# compute compression raios and the PSNR
input_path = 'cat.jpg'
output_path = ['output/cat_n2m4_b.jpg', 'output/cat_n2m8_b.jpg', 
               'output/cat_n4m4_b.jpg', 'output/cat_n4m8_b.jpg']
parameters = [[2, 4], [2, 8], [4, 4], [4, 8]]

for i in range(4):
    n, m = parameters[i]
    ratio = getsize(input_path) / getsize(output_path[i])
    before_bits = 512 * 512 * (8 * 3)
    after_bits = (512/8 * 512/8 * (n**2) * m) + (256/8 * 256/8 * (n**2) * m * 2)
    print('compression ratio (n', n, 'm', m, ') =', before_bits/after_bits)
    print('PSNR = ', PSNR(input_path, output_path[i]))

compression ratio (n 2 m 4 ) = 64.0
PSNR =  20.028385755279036
compression ratio (n 2 m 8 ) = 32.0
PSNR =  20.962387348497327
compression ratio (n 4 m 4 ) = 16.0
PSNR =  22.241875252039215
compression ratio (n 4 m 8 ) = 8.0
PSNR =  27.087264808105537
